In [482]:
import pandas as pd
import numpy as np
import os
import re
# importing required modules 
from tqdm.notebook import tqdm
from pdfminer import high_level
import requests
import ast  # for string to list: ast.literal_eval()
from bs4 import BeautifulSoup

In [205]:
resp = requests.get("https://www.parlament.hu/orszaggyulesi-naplo")

In [211]:
soup = BeautifulSoup(resp.content,"html.parser")

In [218]:
link_list = ["https://www.parlament.hu/" + file.get("href") for file in soup.select("tbody a")]

In [231]:
link_list[176]

'https://www.parlament.hu//documents/10181/1569934/ny180604/51230b5b-cc93-14b2-86aa-d59bf33283b6'

In [233]:
link_list[176].split("/")[7] + ".pdf"

'ny180604.pdf'

In [240]:
for link in tqdm(link_list):
    response = requests.get(link)
    
    with open("naplok/" + link.split("/")[7], "wb") as f:
        f.write(response.content)

In [487]:
os.listdir("naplok/")

['ny190401.pdf',
 'ny180612.pdf',
 'ny200928.pdf',
 'ny190617.pdf',
 'ny190603.pdf',
 'ny200525.pdf',
 'ny200323.pdf',
 'ny191125.pdf',
 'ny201216.pdf',
 'ny200518.pdf',
 'ny201116-ossze.pdf',
 'ny190402.pdf',
 'ny180611.pdf',
 'ny180605.pdf',
 'ny201006-ossze.pdf',
 'ny200615-ossze.pdf',
 'ny200309.pdf',
 'ny180604.pdf',
 'ny180628.pdf',
 'ny190611.pdf',
 'ny181126.pdf',
 '.DS_Store',
 'ny191122.pdf',
 'ny201012.pdf',
 'ny200324.pdf',
 'ny181127.pdf',
 'ny201019-ossze.pdf',
 'ny180629.pdf',
 'ny200519-ossze.pdf',
 'ny190612.pdf',
 'ny200520.pdf',
 'ny181119.pdf',
 'ny201207.pdf',
 'ny191121.pdf',
 'ny201005.pdf',
 'ny190613.pdf',
 'ny190304.pdf',
 'ny201102.pdf',
 'ny200220.pdf',
 'ny200502-unnepiules.pdf',
 'ny200803.pdf',
 'ny190517.pdf',
 'ny200507-ossze.pdf',
 'ny200427.pdf',
 'ny201117-ossze.pdf',
 'ny190701.pdf',
 'ny201103-ossze.pdf',
 'ny190305.pdf',
 'ny190311.pdf',
 'ny181008.pdf',
 'ny180510.pdf',
 'ny190918.pdf',
 'ny190528.pdf',
 'ny180511.pdf',
 'ny190702.pdf',
 'ny19121

In [441]:
string = high_level.extract_text("ny201103-ossze.pdf")
df = pd.DataFrame(string.split("ELNÖK: "))

In [517]:
def clean_text(df):
    try:
        stopwords = [
            df.loc[0, 0].split("Az ülésen jelen voltak:")[1].split("\n\n")[-4].strip(),
            "-\n",
            "\n",
            "\x0c",
        ]
    except IndexError:
        stopwords = [
            df.loc[0, 0].split("Az ülésen jelen volt:")[1].split("\n\n")[-4].strip(),
            "-\n",
            "\n",
            "\x0c",
        ]

    regex = re.compile(".*?\((.*?)\)")

    hun_letters = {
        "Á": "A",
        "É": "E",
        "Ő": "O",
        "Ö": "O",
        "Ó": "O",
        "Ü": "U",
        "Ú": "U",
        "Ű": "U",
    }

    df["name_list"] = None
    df["date"] = stopwords[0]

    for i in tqdm(range(df.shape[0])):

        # basic cleaner
        for stopword in stopwords:
            df.loc[i, 0] = df.loc[i, 0].replace(stopword, "").strip()

        # delete comments
        result = re.findall(regex, df.loc[i, 0])
        result.append("()")

        for res in result:
            df.loc[i, 0] = df.loc[i, 0].replace(res, "").strip()

        # hungarian ÁÉÖŐÓÜÚŰ to AEOU
        for letter in hun_letters.keys():
            df.loc[i, 0] = df.loc[i, 0].replace(letter, hun_letters[letter])

        # extract name list
        df.loc[i, "name_list"] = [
            item.strip()
            for item in re.findall("[A-Z\s]+", df.loc[i, 0])
            if len(item.strip()) > 6
        ]

    df = df.loc[df["name_list"].apply(len) != 0].iloc[1:-1, :].reset_index(drop=True)

    df["name"] = df["name_list"].apply(lambda x: x[0])

    df["text"] = df.apply(lambda x: x[0].split(x["name"])[-1].strip(), axis=1)

    for i in range(df.shape[0]):
        if df.loc[i, "text"][0] == ":":
            df.loc[i, "text"] = df.loc[i, "text"][1:].strip()
        else:
            df.loc[i, "text"] = " ".join(df.loc[i, "text"].split(":")[1:])

    df["len"] = df["text"].apply(len)
    df = df.rename(columns={0: "raw"})

    return df

In [518]:
file_list = ["naplok/" + string for string in os.listdir("naplok/")]
file_list.remove(".DS_Store")

In [519]:
df_concated = pd.DataFrame()
for file in file_list:
    string = high_level.extract_text(file)
    df = pd.DataFrame(string.split("ELNÖK: "))
    df = clean_text(df)

    df_concated = pd.concat([df_concated, df])

PDFSyntaxError: No /Root object! - Is this really a PDF?

In [524]:
file_list

['naplok/ny190401.pdf',
 'naplok/ny180612.pdf',
 'naplok/ny200928.pdf',
 'naplok/ny190617.pdf',
 'naplok/ny190603.pdf',
 'naplok/ny200525.pdf',
 'naplok/ny200323.pdf',
 'naplok/ny191125.pdf',
 'naplok/ny201216.pdf',
 'naplok/ny200518.pdf',
 'naplok/ny201116-ossze.pdf',
 'naplok/ny190402.pdf',
 'naplok/ny180611.pdf',
 'naplok/ny180605.pdf',
 'naplok/ny201006-ossze.pdf',
 'naplok/ny200615-ossze.pdf',
 'naplok/ny200309.pdf',
 'naplok/ny180604.pdf',
 'naplok/ny180628.pdf',
 'naplok/ny190611.pdf',
 'naplok/ny181126.pdf',
 'naplok/.DS_Store',
 'naplok/ny191122.pdf',
 'naplok/ny201012.pdf',
 'naplok/ny200324.pdf',
 'naplok/ny181127.pdf',
 'naplok/ny201019-ossze.pdf',
 'naplok/ny180629.pdf',
 'naplok/ny200519-ossze.pdf',
 'naplok/ny190612.pdf',
 'naplok/ny200520.pdf',
 'naplok/ny181119.pdf',
 'naplok/ny201207.pdf',
 'naplok/ny191121.pdf',
 'naplok/ny201005.pdf',
 'naplok/ny190613.pdf',
 'naplok/ny190304.pdf',
 'naplok/ny201102.pdf',
 'naplok/ny200220.pdf',
 'naplok/ny200502-unnepiules.pdf',
 '

In [528]:
df_concated.loc[df_concated["len"]<1000].shape

(447, 6)

In [529]:
df_concated.shape

(2482, 6)

In [534]:
df_concated["name"].value_counts().head(20)

RETVARI  BENCE              126
TALLAI ANDRAS                72
KERESZTES LASZLO LORANT      57
GULYAS  GERGELY              50
ORBAN  BALAZS                48
VOLNER PAL                   46
TORDAI BENCE                 44
KARPAT DANIEL                43
DOMOTOR  CSABA               43
DAMM ANDREA                  41
ORBAN VIKTOR                 34
SCHMUCK  ERZSEBET            34
BANGONE  BORBELY  ILDIKO     34
CSERESNYES PETER             33
UNGAR PETER                  31
SCHANDA  TAMAS  JANOS        30
CAKE OLIVIO                  28
GURMAI ZITA                  26
NACSA  LORINC                26
KOROZS  LAJOS                26
Name: name, dtype: int64

In [535]:
df_concated

,raw,name_list,date,name,text,len
0,Tisztelt Országgyűlés! Az Országgyűlés tavaszi...,[TOTH BERTALAN],Az Országgyűlés tavaszi ülésszakának 12. ülésn...,TOTH BERTALAN,"Köszönöm a szót, elnök úr. Tisztelt Képviselő...",4759
1,A kormány nevében Dömötör Csaba államtitkár úr...,[DOMOTOR CSABA],Az Országgyűlés tavaszi ülésszakának 12. ülésn...,DOMOTOR CSABA,"Köszönöm a szót, elnök úr. Tisztelt Képviselő...",4340
2,"Szomorúan konstatálom, hogy ellenzéki képvisel...",[GYURCSANY FERENC],Az Országgyűlés tavaszi ülésszakának 12. ülésn...,GYURCSANY FERENC,"Tekintettel arra, hogy a kormány évek óta lén...",3798
3,Ismét Dömötör Csaba államtitkár úré a szó. D...,[DOMOTOR CSABA],Az Országgyűlés tavaszi ülésszakának 12. ülésn...,DOMOTOR CSABA,"Köszönöm a szót, elnök úr. Tisztelt Frakcióv...",1629
4,"Fogják már vissza magukat, képviselőtársaim! ...",[DOMOTOR CSABA],Az Országgyűlés tavaszi ülésszakának 12. ülésn...,DOMOTOR CSABA,"Szóval, ezt a jelentést a Demokratikus Koalíc...",2916
...,...,...,...,...,...,...
183,"Köszönjük, képviselő úr. Következő napirend u...",[ANDER BALAZS],Az Országgyűlés őszi ülésszakának 22. ülésnapj...,ANDER BALAZS,"Köszönöm a szót, elnök úr. Tisztelt Képviselőt...",4670
184,"Köszönjük, képviselő úr. A következő napire...",[KARPAT DANIEL],Az Országgyűlés őszi ülésszakának 22. ülésnapj...,KARPAT DANIEL,Tisztelt Elnök Ur! Tisztelt Ház! „Meddig még?”...,5226
185,"Köszönjük, képviselő úr. A következő napire...",[HORVATH LASZLO],Az Országgyűlés őszi ülésszakának 22. ülésnapj...,HORVATH LASZLO,"Köszönöm a szót, elnök úr. Tisztelt Ház! Aligh...",4301
186,"Köszönjük, képviselő úr. A következő napire...",[UNGAR PETER],Az Országgyűlés őszi ülésszakának 22. ülésnapj...,UNGAR PETER,Köszönöm a szót. Tisztelt Elnök Ur! En lev...,5286


In [513]:
df.loc[0, 0].split("Az ülésen jelen voltak:")

['   2018-2022. országgyűlési ciklus  Budapest, 2020. szeptember 28. hétfő \n\n152. szám \n\n \n\nOrszággyűlési Napló \n\nDr. Latorcai János és Lezsák Sándor  \nelnöklete alatt \n \nJegyzők: dr. Aradszki András, dr. Varga László \n\n \n \n \n \n\n \n\n \n\n \n \n \n \n\n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n\n\x0cHasáb \n\nTárgyai   \n \nAz ülésnap megnyitása ................................................................................................................................ 21319 \n \nNapirend előtti felszólalások: \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\nTordai Bence (Párbeszéd) ...................................................................................................................... 21319 \n \nSchanda Tamás János innovációs és technológiai minisztériumi államtitkár ....................... 21321 \nDr. Keresztes László Lóránt (LMP) ................................................................................................. 21322 \n \n

In [511]:
print(string)

   2018-2022. országgyűlési ciklus  Budapest, 2020. szeptember 28. hétfő 

152. szám 

 

Országgyűlési Napló 

Dr. Latorcai János és Lezsák Sándor  
elnöklete alatt 
 
Jegyzők: dr. Aradszki András, dr. Varga László 

 
 
 
 

 

 

 
 
 
 

 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 

Hasáb 

Tárgyai   
 
Az ülésnap megnyitása ................................................................................................................................ 21319 
 
Napirend előtti felszólalások: 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

Tordai Bence (Párbeszéd) ...................................................................................................................... 21319 
 
Schanda Tamás János innovációs és technológiai minisztériumi államtitkár ....................... 21321 
Dr. Keresztes László Lóránt (LMP) ................................................................................................. 21322 
 
Dr. Rétvári Bence, az Emberi Erőforrások Minisztériumának államtitkára ........

In [ ]:
"................"